# Matrix Multiplication parallelization example 

In [35]:
%%writefile pool_example.py
import numpy as np
import time
from random import random
from multiprocessing import Pool
from functools import partial

def explicit_matmul(A,B):
    #A[m][n]
    #B[n][p]
    #C[m][p]    
    C_temp = [[0 for x in range(np.shape(A)[0])] for y in range(np.shape(B)[1])]
    for i in range(np.shape(A)[0]): #(i=1...m) Rows in A
        for j in range(np.shape(B)[1]): # (j=1...p) Columns in B
            for k in range(np.shape(A)[1]): # (k=1...n) Columns in A
                C_temp[i][j] += A[i][k] * B[k][j]
    return(C_temp)
 
def explicit_matmul_process(A,B,C,k):
    #A[m][n]
    #B[n][p]
    #C[m][p]
    C_temp = [[0 for x in range(np.shape(A)[0])] for y in range(np.shape(B)[1])]
    for i in range(np.shape(A)[0]): #(i=1...m) Rows in A
        for j in range(np.shape(B)[1]): # (j=1...p) Columns in B
            #for k in range(np.shape(A)[1]): # (k=1...n) Columns in A
            C_temp[i][j] += A[i][k] * B[k][j]
    return(C_temp)
 
if __name__ == '__main__':
     
    nprocs= 3
    print(nprocs)
     
    AX=AY=BX=BY=150
    print("Making Matricies")
    
    A = [[random() for x in range(AX)] for y in range(AY)]
    B = [[random() for x in range(BX)] for y in range(BY)]
    C = [[0 for x in range(np.shape(A)[0])] for y in range(np.shape(B)[1])]

    print("starting procs")
    start_thread = time.perf_counter()
    
    ##################################################################
    ##################################################################
    ##################################################################
    ## this specific example is trying to get you to use the pool.map method
    
    #1: You need to create a pool of process
    
    pool = Pool(nprocs)

    #2: Create an iterable for the processes to work on
    
    iter_procs = [i for i in range(AX)]

    #3: Use the partial method from functools to create a "new" function with only 1 Iterable argument!
    ###   Partial allow us to fix a certain number of arguments of a function and generate a new function.
    ###   WE need this because pool.map() only excepts a function and an iterable argument for that function
                
    new_function = partial(explicit_matmul_process,A,B,C)
                           
    #4: Start the processes using the pool.map()
    
    results = pool.map(new_function,iter_procs)

    #5: Make sure no more processes are submitted to pool
    pool.close()
    
    #6: Wait for the worker processes to finish  
    pool.join()
                           
    #7: Make sure results are put back to gether and correct!!!
    ### this is the more difficult step
    ### HINT: use np.sum!
                           
    C_parallel = np.sum(results,axis=0)
    
    ##################################################################
    ##################################################################
    ##################################################################
    
    end_thread = time.perf_counter()
    print("Thread mult: ",end_thread -start_thread)
 
    start_explicit = time.perf_counter()
    C_explicit = explicit_matmul(A,B)
    end_explicit  = time.perf_counter()
    print("Explicit mult: ",end_explicit - start_explicit)
     
    start_np = time.perf_counter()
    C_np = np.matmul(A,B)
    end_np  = time.perf_counter()
    print("NumPy  matmul: ",end_np -start_np)
     
    if not np.allclose(C_explicit, C_parallel, rtol=1e-10, atol=1e-10):
        print("C_parallel is not equal to C_explicit!!")
    if not np.allclose(C_parallel, C_np, rtol=1e-10, atol=1e-10):
        print("C_parallel is not equal to C_np!!")
    if not np.allclose(C_explicit, C_np, rtol=1e-10, atol=1e-10):
        print("C_np is not equal to C_explicit!!")

Overwriting pool_example.py


In [ ]:
!python pool_example.py

3
Making Matricies
starting procs


### Submit a job to the batch system if you want to use more that 3 cores!

In [10]:
%%writefile python_job.sh
#!/bin/bash
#SBATCH -n 16
#SBATCH -p normal
#SBATCH -t 00:30:00


module load 2021 SciPy-bundle/2021.05-foss-2021a

python pool_example.py



Overwriting python_job.sh


In [11]:
!sbatch python_job.sh

Submitted batch job 8957934
